In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Smit\\Desktop\\DESKTOP\\Github Repos\\DS_project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Smit\\Desktop\\DESKTOP\\Github Repos\\DS_project'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainingConfig:
    root_dir: Path 
    train_path: Path
    test_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [16]:
from src.DS_project.utils.common import *
from src.DS_project.constants import *

In [21]:
class ConfigurationManager:
    def __init__(self , 
                 config_path =CONFIG_FILE_PATH , 
                 params_path =PARAMS_FILE_PATH , 
                 schema_path=SCHEMA_FILE_PATH):
        self.config_path = read_yaml(config_path)
        self.params_path = read_yaml(params_path)
        self.schema_path = read_yaml(schema_path)

        create_directories([self.config_path.artifacts_root])

    def get_model_training(self) -> ModelTrainingConfig:
        config = self.config_path.model_training
        create_directories([config.root_dir])
        params = self.params_path.ElasticNet
        schema = self.schema_path.TARGET_COLUMN

        model_training = ModelTrainingConfig(
            root_dir=config.root_dir,
            train_path=config.train_path,
            test_path=config.test_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )

        return model_training

In [22]:
from src.DS_project import logger
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib

In [23]:
class Model_Training:
    def __init__(self , config: ModelTrainingConfig):
        self.config = config

    def train_model(self):

        train_data = pd.read_csv(self.config.train_path)
        test_data = pd.read_csv(self.config.test_path)


        train_x = train_data.drop(self.config.target_column , axis=1)
        train_y = train_data[self.config.target_column]
        test_x = test_data.drop(self.config.target_column , axis =1)
        test_y = test_data[self.config.target_column]

        lr = ElasticNet(alpha=self.config.alpha , l1_ratio=self.config.l1_ratio , random_state=42)

        lr.fit(train_x , train_y)

        joblib.dump(lr , os.path.join(self.config.root_dir , self.config.model_name))



        


In [24]:
try:
    cm = ConfigurationManager()
    model_training_config = cm.get_model_training()
    model_training = Model_Training(config=model_training_config)
    model_training.train_model()
except Exception as e:
    logger.exception(e)
    raise e

2025-07-03 10:40:49,636: INFO: common: Yaml file: config\config.yaml is loaded successfully
2025-07-03 10:40:49,639: INFO: common: Yaml file: params.yaml is loaded successfully
2025-07-03 10:40:49,652: INFO: common: Yaml file: schema.yaml is loaded successfully
2025-07-03 10:40:49,655: INFO: common: Directory artifacts is created
2025-07-03 10:40:49,657: INFO: common: Directory artifacts/model_training is created
